In [5]:
import xgboost as xgb
import pandas as pd
import numpy as np
import scipy as sc

In [14]:
train_set = pd.read_csv('dataset/output/train_local_dataset_polimi_extended_only_2016.csv',delimiter=',')
test_set = pd.read_csv('dataset/output/test_local_dataset_polimi_extended_no2014.csv',delimiter=',')

In [15]:
y_train = train_set.pop('vendite')
t_train = test_set.pop('vendite')

In [16]:
xgdmat = xgb.DMatrix(train_set,y_train, missing=np.nan)
dtest = xgb.DMatrix(test_set, t_train, missing=np.nan)

In [17]:
def mape(predictions, dtrain):
    ground_truth = dtrain.get_label()
    one_over_n = 1/len(predictions)
    at_mean = sum(ground_truth)/len(ground_truth)
    sommatoria = sum(np.abs(ground_truth - predictions)) / at_mean
    return 'mape', one_over_n * sommatoria

In [35]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':10, 'min_child_weight':10} 

# Questo dictionary verra popolato con tutti i risultati della MAPE del train e 
# test set computati ad ogni iterazione
dic = {}

# Ad ogni iteraziore, voglio 'guardare' questi due dataset e valutarli con la funzione
# passata in feval
watchlist = [(xgdmat,'train'),(dtest,'test')]

final_gb = xgb.train(our_params, xgdmat, 4000, evals = watchlist, feval = mape, evals_result = dic)

[0]	train-mape:0.824798	test-mape:0.809217
[1]	train-mape:0.750642	test-mape:0.725894
[2]	train-mape:0.6846	test-mape:0.652896
[3]	train-mape:0.629362	test-mape:0.593164
[4]	train-mape:0.57999	test-mape:0.540414
[5]	train-mape:0.539486	test-mape:0.49813
[6]	train-mape:0.504515	test-mape:0.464633
[7]	train-mape:0.471328	test-mape:0.432081
[8]	train-mape:0.440891	test-mape:0.404932
[9]	train-mape:0.413571	test-mape:0.376502
[10]	train-mape:0.392012	test-mape:0.356458
[11]	train-mape:0.373155	test-mape:0.339302
[12]	train-mape:0.355365	test-mape:0.322679
[13]	train-mape:0.340214	test-mape:0.310278
[14]	train-mape:0.328164	test-mape:0.300636
[15]	train-mape:0.31551	test-mape:0.289996
[16]	train-mape:0.306264	test-mape:0.285342
[17]	train-mape:0.300415	test-mape:0.283259
[18]	train-mape:0.292926	test-mape:0.279361
[19]	train-mape:0.284105	test-mape:0.270678
[20]	train-mape:0.278921	test-mape:0.266233
[21]	train-mape:0.273489	test-mape:0.261229
[22]	train-mape:0.268418	test-mape:0.257237
[23

KeyboardInterrupt: 

In [32]:
min(dic['train']['mape'])

0.070677

In [33]:
min(dic['test']['mape'])

0.064618

In [30]:
# Prediction sul test set
testdmat = xgb.DMatrix(test_set, missing=np.nan)
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred[:10]

array([ 7.90523338,  8.25818443,  8.63885117,  7.39155674,  0.38436493,
        0.25864041,  9.90960026,  7.98770618,  8.33055496,  4.42607403], dtype=float32)

In [34]:
# Prediction sul test set
testdmat = xgb.DMatrix(test_set, missing=np.nan)
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred[:10]

array([  8.17926598,   8.17908859,   9.56858349,   7.07249832,
         0.19842401,   0.1627059 ,  10.00421619,   7.76880169,
         8.78395271,   4.25869083], dtype=float32)

In [51]:
def mape_noxgb(predictions, dtrain):
    ground_truth = dtrain#.get_label()
    one_over_n = 1/len(predictions)
    at_mean = sum(ground_truth)/len(ground_truth)
    sommatoria = sum(np.abs(ground_truth - predictions)) / at_mean
    return 'mape', one_over_n * sommatoria

In [54]:
round_pred = list(map(lambda x: round(x), y_pred))
mape_noxgb(round_pred,t_train)

('mape', 0.025537904685300625)